In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('VPA_Open_Space.csv')
selected = ['LGA', 'PARK_NAME', 'OS_CATEGOR', 'OS_ACCESS', 'VM_PARCE_1']
selected_df = data[selected]

In [3]:
limited_category = selected_df[(selected_df['OS_CATEGOR'] != 'Transport reservations') &
                              (selected_df['OS_CATEGOR'] != 'Services and utilities reserves') &
                              (selected_df['OS_CATEGOR'] != 'Non-government schools') &
                              (selected_df['OS_CATEGOR'] != 'Government schools') &
                              (selected_df['OS_CATEGOR'] != 'Civic squares and promenades') &
                              (selected_df['VM_PARCE_1'] != 'NO DATA')] # These rows don't have valid Park name

limited_category['OS_CATEGOR'].mask(limited_category['OS_CATEGOR'] == 'Recreation corridor', 'Recreational Reserve', inplace = True)

limited_category['OS_CATEGOR'].mask(limited_category['OS_CATEGOR'] == 'Natural and semi-natural open space', 'Natural attraction (reserves)', inplace = True)

limited_category['OS_CATEGOR'].mask(limited_category['OS_CATEGOR'] == 'Conservation reserves', 'Natural attraction (reserves)', inplace = True)


D:\anaconda3\lib\site-packages\pandas\core\generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


### Some Park names are incorrect (e.g. Gray, J.T. Reserve should be J.T. Gray Reserve)

In [4]:
open_park = limited_category[limited_category['OS_ACCESS'] == 'Open']

remove_duplicated_park = open_park.drop_duplicates(keep='first')

# drop empty string or white space row
remove_empty_park = remove_duplicated_park[remove_duplicated_park['PARK_NAME'].str.strip().astype(bool)]

In [5]:
remove_empty_park.reset_index(drop=True,inplace=True)

import numpy as np
remove_empty_park['PARK_NAME'].replace('', np.nan, inplace=True)
remove_empty_park.dropna(subset=['PARK_NAME'],inplace=True)
#remove_empty_park.dropna(how='any',inplace=True)
remove_empty_park.reset_index(drop=True,inplace=True)

D:\anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-5-16fd542e3024>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remove_empty_park.dropna(subset=['PARK_NAME'],inplace=True)


In [6]:
import re
pattern = re.compile(r'\,\s\w\.\w\.\s[\w\W]+')
test=[]
for idx,row in remove_empty_park.iterrows():
    if re.findall(pattern, row['PARK_NAME']):
        test.append(row['PARK_NAME'])
test

['Bond, A.W. Reserve',
 'Ginifer, J.J. Reserve',
 'Hearn, V.L. Reserve',
 'Ford, A.H. Reserve',
 'Gray, J.T. Reserve',
 'Richards, R.K. Reserve',
 'Ransom, G.H. Reserve',
 'Irwin, W.J. Reserve',
 'Langshaws, A.W. Reserve',
 'Stirling, G.F. Reserve',
 'Emery, F.A. Reserve',
 'Hosken, G.J. Reserve',
 'Hosken, G.J. Reserve',
 'Cooper, R.J. Reserve',
 'McIntosh, S.G. Reserve',
 'O`Brien, J.F. Reserve',
 'Grant, J.K. Reserve',
 'Lee, H.W. Reserve',
 'Stewart, J.J. Reserve',
 'Carlsson, C.W. Reserve',
 'Knight Reserve, A.W. (Altona Meadows)',
 'Hubble, W.E. Reserve',
 'Kim, H.C. Reserve',
 'Cresser, W.G. Reserve',
 'Ginifer, J.J. Reserve',
 'Ginifer, J.J. Reserve']

#### Change these Park Name to correct format (e.g. Gray, J.T. Reserve should be J.T. Gray Reserve)

In [7]:
pattern = re.compile(r'\,\s\w\.\w\.\s[\w\W]+') # this pattern matches strings like 'O`Brien, J.F. Reserve'

for idx,row in remove_empty_park.iterrows():
    remove_empty_park = remove_empty_park.copy() # prepare to change dataframe inplace
    if re.findall(pattern, row['PARK_NAME']):
        name_list = row['PARK_NAME'].split(', ') # split strings like 'O`Brien, J.F. Reserve' to ['O`Brien', 'J.F. Reserve']
        temp = name_list[1].split('. ') # split strings like 'A.W. (Altona Meadows)' to ['A.W', '(Altona Meadows)']
        
        # rename strings like 'Gray, J.T. Reserve' to 'J.T. Gray Reserve'
        remove_empty_park.at[idx, 'PARK_NAME'] = temp[0] + ". " + name_list[0] + ' ' + temp[1] # rename

In [8]:
pattern = re.compile(r'\,\s\w\.\s\w+')
test=[]
for idx,row in remove_empty_park.iterrows():
    if re.findall(pattern, row['PARK_NAME']):
        test.append(row['PARK_NAME'])
test

['Frazer, R. Reserve', 'Grieve, D. Reserve', 'Rush, F. Reserve']

#### There are some incorrect park names such as 'Frazer, R. Reserve'
#### Rename these names

In [9]:
pattern = re.compile(r'\,\s\w\.\s\w+') # this pattern matches strings like 'Frazer, R. Reserve'

for idx,row in remove_empty_park.iterrows():
    remove_empty_park = remove_empty_park.copy() # prepare to change dataframe inplace
    if re.findall(pattern, row['PARK_NAME']):
        name_list = row['PARK_NAME'].split(', ') # split strings like 'Frazer, R. Reserve' to ['Frazer', 'R. Reserve']
        temp = name_list[1].split() # split strings like 'R. Reserve' to ['R.', 'Reserve']
        
        # rename strings like 'Gray, J.T. Reserve' to 'J.T. Gray Reserve'
        remove_empty_park.at[idx, 'PARK_NAME'] = temp[0] + " " + name_list[0] + ' ' + temp[1] # rename

In [10]:
# There is only one entry like Crofts, W.L.J. Reserve, hence we change it by mask() to save calculation
remove_empty_park['OS_CATEGOR'].mask(remove_empty_park['PARK_NAME'] == 
                                   'Crofts, W.L.J. Reserve', 'W.L.J. Crofts Reserve', inplace = True)


### Some Park_Name contains park address. Extract the address and store in a new column

In [11]:
# create a list to store idx and park_name pair
park_name_list = []

for idx, row in remove_empty_park.iterrows():
    if ',' in row['PARK_NAME']:
        park_list = row['PARK_NAME'].split(',')
        temp = []
        temp.append(idx)
        temp.append(park_list)
        
        park_name_list.append(temp)

In [12]:
park_name_list

[[34, ['Drains', ' Hawthorn']],
 [60, ['Trunk Sewer', ' South Eastern']],
 [86, ['Retarding Basins', ' Hawthorn East']],
 [88, ['Burke Road Billabong Reserve', ' Kew East']],
 [334, ['Burke Road Billabong Reserve', ' Kew East']],
 [352, ['Drains', ' Ashburton (Camberwell)']],
 [389, ['Trunk Sewer', ' South Eastern']],
 [414, ['Drains', ' Hawthorn']],
 [452, ['Drains', ' Ashburton (Camberwell)']],
 [459, ['350a Tooronga Rd', ' Hawthorn']],
 [582, ['Flood Plains', ' Improvement - Taylors Creek']],
 [595, ['Reserve', ' Manningham', ' Drainage']],
 [607, ['Reserve', ' Manningham', ' Drainage']],
 [635, ['Flood Plains', ' Improvement - Ruffeys Creek']],
 [753, ['Retarding Basins', ' Wetherby Road (Doncaster)']],
 [762, ['Flood Plains', ' Yarra River']],
 [765, ['Retarding Basins', ' Wetherby Road (Doncaster)']],
 [798, ['Flood Plains', ' Improvement - Ruffeys Creek']],
 [800, ['Retarding Basins', ' Andersons Creek']],
 [802, ['Flood Plains', ' Improvement - Ruffeys Creek']],
 [803, ['Drains

#### There is one record '350a Tooronga Rd, Hawthorn' and when I google it, I found that this is actually not any parks. It's actually a residence community. Hence I believe this is a wrong entry and I would remove this record

In [13]:
park_name_list.remove([459, ['350a Tooronga Rd', ' Hawthorn']])
remove_empty_park.drop(index=459,inplace=True)
remove_empty_park.reset_index(drop=True,inplace=True)

### If the length of sublist is more than 2,  the record doesn't contain meaningful address.
#### Most of the records with length of sublist equals 2 contain meaningful address (e.g. [5014, ['Mullum Creek Reserve 4', ' 18 Reserve Road']]). 
#### Hence we keep these rows and try to remove the sublist of sublist[1] that doesn't contain space (e.g. [82, ['Drains', ' Hawthorn']].

#### There are some address contains ' Improvement - '. We need to remove these.

In [14]:
# remove the 'NO DATA' rows
remove_empty_park.drop(remove_empty_park[remove_empty_park['PARK_NAME'] == 'NO DATA'].index, inplace=True)
remove_empty_park.reset_index(drop=True,inplace=True)



In [15]:
# create a list to store idx and park_name pair
park_name_list = []

for idx, row in remove_empty_park.iterrows():
    if ',' in row['PARK_NAME']:
        park_list = row['PARK_NAME'].split(',')
        temp = []
        temp.append(idx)
        temp.append(park_list)
        
        park_name_list.append(temp)

In [16]:
remove_empty_park['ADDRESS'] = None


for lists in park_name_list:
    remove_empty_park.at[lists[0],'PARK_NAME'] = lists[1][0]
    if (len(lists[1]) == 2) & (' ' in lists[1][1].strip()):
        if 'Improvement - ' in lists[1][1]:
            name_without_improvement = lists[1][1].replace('Improvement - ', '')
            remove_empty_park.at[lists[0],'ADDRESS'] = name_without_improvement
        elif 'improvement - ' in lists[1][1]:
            name_without_improvement = lists[1][1].replace('improvement - ', '')
            remove_empty_park.at[lists[0],'ADDRESS'] = name_without_improvement
        else:
            remove_empty_park.at[lists[0],'ADDRESS'] = lists[1][1]

## There are some entries with len(PARK_NAME) = 1, which are some places that aren't parks. So I'll remove these.

## When you do len(df['column name']) you are just getting one number, namely the number of rows in the DataFrame (i.e., the length of the column itself). If you want to apply len to each element in the column, use df['column name'].map(len)

In [17]:
remove_empty_park[remove_empty_park['PARK_NAME'].str.split().map(len) < 2]

,LGA,PARK_NAME,OS_CATEGOR,OS_ACCESS,VM_PARCE_1,ADDRESS
34,BOROONDARA,Drains,Natural attraction (reserves),Open,2058438,None
352,BOROONDARA,Drains,Natural attraction (reserves),Open,805290,Ashburton (Camberwell)
414,BOROONDARA,Drains,Natural attraction (reserves),Open,221356880,None
452,BOROONDARA,Drains,Natural attraction (reserves),Open,817474,Ashburton (Camberwell)
594,MANNINGHAM,Reserve,Natural attraction (reserves),Open,1337602,None
...,...,...,...,...,...,...
17939,FRANKSTON,Drains,Natural attraction (reserves),Open,80074,Wadsleys Diversion
17941,FRANKSTON,Drains,Natural attraction (reserves),Open,207394923,Wadsleys Diversion
17943,FRANKSTON,Drains,Natural attraction (reserves),Open,208012880,Wadsleys Diversion
17946,FRANKSTON,Drains,Natural attraction (reserves),Open,202832813,Wadsleys Diversion


In [18]:
remove_empty_park.drop(remove_empty_park[remove_empty_park['PARK_NAME'].str.split().map(len) < 2].index, inplace=True)
remove_empty_park.reset_index(drop=True,inplace=True)

## There are some entries that named 'Trunk Sewer' under PARK_NAME. This is likely a wrong entry because when we search 'Trunk Sewer' on google it returns sewerage. Hence we remove these records

In [19]:
remove_empty_park.drop(remove_empty_park[remove_empty_park['PARK_NAME'] == 'Trunk Sewer'].index, inplace=True)
remove_empty_park.reset_index(drop=True,inplace=True)

#### There are some entries that have 'improvement' in PARK_NAME. So we replace the improvement with ''

In [20]:
remove_empty_park['PARK_NAME'] = remove_empty_park['PARK_NAME'].apply(lambda x: x.replace('IMPROVEMENT - ', ''))

## There are some address separated by r'\s\-\s' for example: 
## 'Old Wells Road Tree Reserve - West Section 2 Part A - Kurrawa Crescent (Sth) to Palm Beach Drive - (150m South of Kurrawa Crescent (Sth)'. 
## Extract these address

## There are some address separated by ' -  AKA:' such as:

## 'Farm Road Reserve - AKA: Former Freeway Reservation - Frontage to Centre Dandenong Road, Brampton Street'

## Need to bring the AKA: Former Freeway Reservation back to PARK_NAME

In [21]:
for idx, row in remove_empty_park.iterrows():
    remove_empty_park = remove_empty_park.copy()
    # check if ' - AKA' is in the PARK_NAME
    if ' - AKA' in row['PARK_NAME']:
        name_list = row['PARK_NAME'].split(' - ')
        remove_empty_park.at[idx, 'ADDRESS'] = ' - '.join(name_list[2:]) # concatenate the strings with ' - ' from name_list[2:]
        remove_empty_park.at[idx, 'PARK_NAME'] = ' '.join(name_list[0:2]) # concatenate the strings with ' ' from name_list[0:2]
    else:
        name_list = row['PARK_NAME'].split(' - ')
        remove_empty_park.at[idx, 'ADDRESS'] = ' - '.join(name_list[1:])
        remove_empty_park.at[idx, 'PARK_NAME'] = ''.join(name_list[0])
    

## Concatenate with Swimming pool

In [22]:
swimming_pool = pd.read_csv('swimming pool final.csv')
selected_park = ['LGA', 'PARK_NAME', 'OS_CATEGOR', 'ADDRESS']
park_df = remove_empty_park[selected_park]

park_df.rename(columns={'OS_CATEGOR':'CATEGORY'}, inplace=True)

park_swim_df = pd.concat([park_df,swimming_pool])



D:\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [23]:
track_df = pd.read_csv('tmpcSTzXmtracking.csv', header=None)

In [24]:
track_selected = track_df.drop(columns=[4,6,7])

# rename columns to match other datasets
track_selected.rename(columns={0:'PARK_NAME', 1: 'address_1', 2:'ADDRESS', 3:'POST_CODE', 5:'CATEGORY'}, inplace=True)

In [25]:
track_selected

,PARK_NAME,address_1,ADDRESS,POST_CODE,CATEGORY
0,Australian Alps Walking Track,NaN,Alpine National Park,3825,Tracks/Trails
1,Ballarat-Skipton Railtrail,NaN,Ballarat,3350,Tracks/Trails
2,Barmah Lakes Loop Track,NaN,Barmah State Park and Forest,3639,Tracks/Trails
3,Bayside Trail - Williamstown-Brighton,NaN,Melbourne,3000,Tracks/Trails
4,Beehive Falls Trail,Grampians National Park,Grampians,3381,Tracks/Trails
...,...,...,...,...,...
110,Wilsons Prom Circuit Trail,NaN,Wilsons Promontory National Park,3960,Tracks/Trails
111,Wirrawilla Walk - Toolangi State Forest,NaN,Toolangi State Forest,3777,Tracks/Trails
112,Wonderland Loop Walk,NaN,Grampians National Park,3381,Tracks/Trails
113,Woolshed Falls Historic Walk,NaN,Beechworth Historic Park,3747,Tracks/Trails


In [26]:

for idx, row in track_selected.iterrows():
    track_selected = track_selected.copy()
    if type(row['address_1']) == str:
        track_selected.at[idx, 'ADDRESS'] = row['address_1']


In [27]:
track_selected.drop(columns=['address_1'],inplace=True)

In [28]:
track_selected

,PARK_NAME,ADDRESS,POST_CODE,CATEGORY
0,Australian Alps Walking Track,Alpine National Park,3825,Tracks/Trails
1,Ballarat-Skipton Railtrail,Ballarat,3350,Tracks/Trails
2,Barmah Lakes Loop Track,Barmah State Park and Forest,3639,Tracks/Trails
3,Bayside Trail - Williamstown-Brighton,Melbourne,3000,Tracks/Trails
4,Beehive Falls Trail,Grampians National Park,3381,Tracks/Trails
...,...,...,...,...
110,Wilsons Prom Circuit Trail,Wilsons Promontory National Park,3960,Tracks/Trails
111,Wirrawilla Walk - Toolangi State Forest,Toolangi State Forest,3777,Tracks/Trails
112,Wonderland Loop Walk,Grampians National Park,3381,Tracks/Trails
113,Woolshed Falls Historic Walk,Beechworth Historic Park,3747,Tracks/Trails


In [29]:
track_selected['CATEGORY'].mask(track_selected['CATEGORY'] == 'Tracks/Trails', 'Trails', inplace = True)

In [30]:
locality = pd.read_csv('locality.csv')

## Find LGAs for each postcode

In [31]:
track_post_code = track_selected['POST_CODE'].unique().tolist()

# record LGAs in a list if postcode from track dataset matches postcode from locality
track_lga = []
for code in track_post_code:
    for idx,row in locality.iterrows():
        if row['POST_CODE'] == code:
            temp = []
            temp.append(code)
            temp.append(row['LGA'])
            if temp not in track_lga:
                track_lga.append(temp)

In [32]:
track_selected['LGA'] = ''

for idx,row in track_selected.iterrows():
    track_selected = track_selected.copy()
    lga_string = ''
    for lists in track_lga:
        
        # look for postcodes that are in track_lga lists
        if row['POST_CODE'] == lists[0]:
            lga_string += lists[1] + ',' # replace postcodes with LGA and separate by comma
    
    track_selected.at[idx,'LGA'] = lga_string # change 'LGA' column to LGAs
            

In [33]:
import re
pattern = re.compile(r'\W\B') # patterns like ',' or ';' and etc.

# remove the comma at the end of string (e.g 'BAW BAW,LATROBE,')
for idx, row in track_selected.iterrows():
    track_selected = track_selected.copy()
    track_selected.at[idx, 'LGA'] = re.sub(pattern, '', row['LGA'])

In [34]:
# create a list to store idx and LGA pair
track_lga_list = []

for idx, row in track_selected.iterrows():
    if not row['LGA'].isalpha():
        LGA_list = row['LGA'].split(',')
        temp = []
        temp.append(idx)
        temp.append(LGA_list)
        
        track_lga_list.append(temp)

In [35]:
# false_pairs are something like [25, ['Port Phillip']] where the second element in the list returns False when tested with
# str.isalpha() but actually it is a one-element-list. The reason that caused this is that this element has a white space in
# its string
false_pairs = []
for pairs in track_lga_list:
    if len(pairs[1]) == 1:
        false_pairs.append(pairs)

In [36]:
for pairs in false_pairs:
    
    track_lga_list.remove(pairs)

### create a new dataframe to store all the multiple LGA and corresponding records

In [37]:
track_duplicate_LGA = pd.DataFrame(columns=['LGA', 'PARK_NAME', 'CATEGORY', 'ADDRESS'])

# store data in corresponding lists
track_lga_final_list = []
track_park_name_list = []
track_category_list = []
track_address_list = []
for lists in track_lga_list:
    for lga in lists[1]:
        track_lga_final_list.append(lga)
        track_park_name = track_selected.iloc[lists[0]]['PARK_NAME']
        track_category = track_selected.iloc[lists[0]]['CATEGORY']
        track_address = track_selected.iloc[lists[0]]['ADDRESS']
        
        track_park_name_list.append(track_park_name)
        track_category_list.append(track_category)
        track_address_list.append(track_address)
        
# store data in corresponding columns        
track_duplicate_LGA['LGA'] = track_lga_final_list
track_duplicate_LGA['PARK_NAME'] = track_park_name_list
track_duplicate_LGA['CATEGORY'] = track_category_list
track_duplicate_LGA['ADDRESS'] = track_address_list

### Remove multiple LGA rows from original dataframe in order to concatenate the duplicate LGA dataframe and the single LGA rows dataframe

In [38]:
for pairs in track_lga_list:
    track_selected.drop(pairs[0],inplace=True)
    
track_selected.drop(columns=['POST_CODE'], inplace=True)

track_final_df = pd.concat([track_selected, track_duplicate_LGA])

## There are a few empty string entries in LGA column.
## To drop these rows, use np.nan to replace the empty strings then df.dropna()

In [39]:
import numpy as np
track_final_df['LGA'].replace('', np.nan, inplace=True)
track_final_df.dropna(subset=['LGA'],inplace=True)

In [40]:
track_park_swim = pd.concat([park_swim_df, track_final_df])

track_park_swim.drop_duplicates(keep='first',inplace=True)

track_park_swim.sort_values(by=['LGA','PARK_NAME','CATEGORY'],axis = 0,inplace=True)

#### There are some special characters left behind some address [e.g. 'Billingham)']. We'll remove these records

In [41]:
track_park_swim.reset_index(drop=True,inplace=True)
special_character = []
for idx,row in track_park_swim.iterrows():
    if type(row['ADDRESS']) == str:
        if ')' in row['ADDRESS']:
            temp = []
            temp.append(idx)
            temp.append(row['ADDRESS'])
            special_character.append(temp)
special_character

[[857, 'Billingham)'],
 [858, 'Davitt)'],
 [2165, ' Broadway (Preston)'],
 [4062, 'Seaford Wetlands (MWC drains)'],
 [4215,
  'West Section 2 Part A - Kurrawa Crescent (Sth) to Palm Beach Drive - (150m South of Kurrawa Crescent (Sth)'],
 [4216,
  'West Section 2 Part A - Kurrawa Crescent (Sth) to Palm Beach Drive - (South side Kurrawa Crescent (Sth))'],
 [4217,
  'West Section 1 Part B - Kurrawa Crescent to Kurrawa Crescent - (South side of Kurrawa Cresecent (Nth)) North side of Kurrawa Crescent (North)'],
 [4218,
  'West Section 2 Part A - Kurrawa Crescent (Sth) to Palm Beach Drive - (South side Palm Beach Drive'],
 [4219,
  'West Section 2 Part A - Kurrawa Crescent (Sth) to Palm Beach Drive - (North side Palm Beach Drive)'],
 [4220,
  'West Section 2 Part A - Kurrawa Crescent (Sth) to Palm Beach Drive - (50m South of Kurrawa Crescent (Sth)'],
 [4221,
  'West Section 1 Part C - Kurrawa Crescent to Kurrawa Crescent - (100m South of Kurrawa Cresecent (Nth)'],
 [4326,
  'North Section 1 

In [42]:
track_park_swim.drop(index=[857,858],inplace=True)

In [43]:
track_park_swim.reset_index(drop=True,inplace=True)

In [44]:
track_park_swim

,LGA,PARK_NAME,CATEGORY,ADDRESS
0,ALPINE,Gorge Heritage Walk,Trails,Mt Buffalo National Park
1,ALPINE,Mt Bogong Trail,Trails,Alpine National Park
2,ALPINE,Mt Feathertop via Razorback Trail,Trails,Alpine National Park
3,ALPINE,Mt Hotham Dinner Plain Ski Trail,Trails,Mt Hotham Resort/Alpine National Park
4,ALPINE,Myrtleford Swimming Pool,Swimming pool,Standish Street
...,...,...,...,...
9900,YARRA RANGES,Yarraridge Reserve,Parks and gardens,
9901,YARRA RANGES,Yellingbo Nature Conservation Reserve,Natural attraction (reserves),
9902,YARRIAMBIACK,Desert Discovery Walk,Trails,Little Desert National Park
9903,YARRIAMBIACK,Minyip Swimming Pool,Swimming pool,Foundry Street


#### There are some address repeating LGA names, we will remove these address but keep the records

In [45]:
repeated_address = []
for idx,row in track_park_swim.iterrows():
    track_park_swim = track_park_swim.copy()
    if type(row['ADDRESS']) == str:
        if row['ADDRESS'].upper() == row['LGA']:
            temp = []
            temp.append(idx)
            temp.append(row['ADDRESS'])
            repeated_address.append(temp)
repeated_address

[[21, 'Ballarat'],
 [23, 'Ballarat'],
 [24, 'Ballarat'],
 [25, 'Ballarat'],
 [92, 'Banyule'],
 [93, 'Banyule'],
 [338, 'Banyule'],
 [339, 'Banyule'],
 [514, 'Boroondara'],
 [661, 'Boroondara'],
 [668, 'Boroondara'],
 [780, 'Brimbank'],
 [862, 'Brimbank'],
 [891, 'Brimbank'],
 [1072, 'Cardinia'],
 [1073, 'Cardinia'],
 [1078, 'Cardinia'],
 [1081, 'Cardinia'],
 [1107, 'Cardinia'],
 [1109, 'Cardinia'],
 [1110, 'Cardinia'],
 [1168, 'Cardinia'],
 [1177, 'Cardinia'],
 [1188, 'Cardinia'],
 [1202, 'Cardinia'],
 [1203, 'Cardinia'],
 [1211, 'Cardinia'],
 [1217, 'Cardinia'],
 [1218, 'Cardinia'],
 [1255, 'Cardinia'],
 [1266, 'Cardinia'],
 [1268, 'Cardinia'],
 [1283, 'Cardinia'],
 [1292, 'Cardinia'],
 [1295, 'Cardinia'],
 [1300, 'Cardinia'],
 [1424, 'Casey'],
 [1426, 'Casey'],
 [1495, 'Casey'],
 [1511, 'Casey'],
 [1605, 'Casey'],
 [1774, 'Casey'],
 [1843, 'Casey'],
 [1986, 'Casey'],
 [2034, 'Casey'],
 [2035, 'Casey'],
 [2069, 'Casey'],
 [2137, 'Darebin'],
 [2159, 'Darebin'],
 [2171, 'Darebin'],
 [21

In [46]:
index_list = [lists[0] for lists in repeated_address]
track_park_swim.drop(index=index_list,inplace=True)

In [47]:
track_park_swim

,LGA,PARK_NAME,CATEGORY,ADDRESS
0,ALPINE,Gorge Heritage Walk,Trails,Mt Buffalo National Park
1,ALPINE,Mt Bogong Trail,Trails,Alpine National Park
2,ALPINE,Mt Feathertop via Razorback Trail,Trails,Alpine National Park
3,ALPINE,Mt Hotham Dinner Plain Ski Trail,Trails,Mt Hotham Resort/Alpine National Park
4,ALPINE,Myrtleford Swimming Pool,Swimming pool,Standish Street
...,...,...,...,...
9900,YARRA RANGES,Yarraridge Reserve,Parks and gardens,
9901,YARRA RANGES,Yellingbo Nature Conservation Reserve,Natural attraction (reserves),
9902,YARRIAMBIACK,Desert Discovery Walk,Trails,Little Desert National Park
9903,YARRIAMBIACK,Minyip Swimming Pool,Swimming pool,Foundry Street


In [48]:
track_park_swim.reset_index(drop=True,inplace=True)

In [52]:
bad_parkName = []
for idx,row in track_park_swim.iterrows():
    track_park_swim = track_park_swim.copy()
    if type(row['PARK_NAME']) == str:
        if 'SEC Easement' in row['PARK_NAME']:
            temp = []
            temp.append(idx)
            temp.append(row['PARK_NAME'])
            bad_parkName.append(temp)
bad_parkName

[[939, 'SEC Easement'],
 [940, 'SEC Easement'],
 [941, 'SEC Easement'],
 [942, 'SEC Easement'],
 [943, 'SEC Easement'],
 [944, 'SEC Easement'],
 [945, 'SEC Easement'],
 [946, 'SEC Easement'],
 [947, 'SEC Easement '],
 [985, 'Taylors Lakes SEC Easement (Sunshine Ave'],
 [986, 'Taylors Lakes SEC Easement (Sunshine Ave']]

#### After searching 'Taylors Lakes SEC Easement (Sunshine Ave', 'Taylors Lakes' is actually a suburb. So we remove all these records because they are not parks

In [53]:
index_list = [lists[0] for lists in bad_parkName]
track_park_swim.drop(index=index_list,inplace=True)
track_park_swim.reset_index(drop=True,inplace=True)

In [54]:
bad_address = []
for idx,row in track_park_swim.iterrows():
    track_park_swim = track_park_swim.copy()
    if type(row['ADDRESS']) == str:
        if 'SEC Easement' in row['ADDRESS']:
            temp = []
            temp.append(idx)
            temp.append(row['ADDRESS'])
            bad_address.append(temp)
bad_address

[[837, 'SEC Easement'],
 [838, 'SEC Easement'],
 [839, 'SEC Easement'],
 [918, 'SEC Easement']]

In [55]:
index_list = [lists[0] for lists in bad_address]
track_park_swim.drop(index=index_list,inplace=True)
track_park_swim.reset_index(drop=True,inplace=True)

In [ ]:
# generate LGA_ID
LGA_ID = []
counter = 800000
for i in range(len(track_park_swim)):
    counter += 1
    LGA_ID.append(counter)

In [ ]:
# create final dataframe
final_df = pd.DataFrame({'LGA_ID' : LGA_ID,
                       'LGA' : track_park_swim['LGA'],
                       'PARK_NAME' : track_park_swim['PARK_NAME'],
                       'CATEGORY' : track_park_swim['CATEGORY'],
                        'ADDRESS' : track_park_swim['ADDRESS']})

final_df

In [ ]:
locality_lga = locality['LGA'].sort_values().unique()
final_lga = final_df['LGA'].sort_values().unique()
locality_difference = set(final_lga) - set(locality_lga)
locality_difference

In [ ]:
final_df.to_csv('LGA with park and swim and track.csv', index=False)